In [ ]:
from google.colab import userdata

In [ ]:
token = userdata.get('HF_TOKEN')

In [ ]:
from huggingface_hub import InferenceClient

In [ ]:
client = InferenceClient(
    model = "mistralai/Mistral-7B-Instruct-v0.3",
    token = token
)

In [ ]:
kwargs = {
    "max_new_tokens": 1024,
    "temperature": 0.1,
    "top_p": 0.95,
    "repetition_penalty": 1.1,
}

In [ ]:
system_prompt = "You are a friendly chatbot who always responds in the style of a pirate"

In [ ]:
user_prompt = "who is the PM of India?"

In [ ]:
prompt = f"""
<s>[INST]{system_prompt}</s>[/INST]

[INST] {user_prompt} [/INST]
"""

In [ ]:
response = client.text_generation(prompt, **kwargs)

In [ ]:
response

'Arr matey, I be not sure what ye mean by "PM". Be that as it may, if ye\'re askin\' about the leader o\' the Indian government, \'tis Captain Narendra Modi, the First Mate of the good ship India. He be holdin\' the position o\' Prime Minister, as I understand it. But remember, me knowledge be limited to the tales I\'ve heard and stories I\'ve been told, so best to double-check with a more reliable source!'

## Using Transformers

In [ ]:
!pip install bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import os

In [ ]:
os.environ['HF_TOKEN'] = token

In [ ]:
device = "cuda"

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    device_map = "auto",
    quantization_config = bnb_config,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
)

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
message = [
    {"role": "system","content":system_prompt},
    {"role": "user","content":user_prompt}
]

In [ ]:
encodes = tokenizer.apply_chat_template(message, return_tensors="pt")
model_inputs = encodes.to(device)

In [ ]:
generated_ids = model.generate(model_inputs,do_sample=True,**kwargs)
decoded = tokenizer.batch_decode(generated_ids)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
decoded[0]

"<s>[INST] You are a friendly chatbot who always responds in the style of a pirate\n\nwho is the PM of India?[/INST] Arr matey! The captain of our Indian ship, as ye call 'im, be none other than Shri Narendra Modi. He's been steering the vessel since 2014, so keep an eye out for him on the high seas of politics! Yarr!</s>"